# LDA

- Kmeansとの違い
    - ハードクラスタリングとソフトクラスタリング
    - Kmeansが距離によるクラスタリングに対して、潜在変数のディレクレ分布を推定
- 理論
    - 単語では見えない背後にある意味を分類する
    - 単語ごとに各潜在変数を背後に持っていて、その潜在変数の値が同じ場合に同じトピックに属する
    - 単語出現確率は、POSでいうところの1回の買い物で買う確率
    - ディリクレ分布を用いて、潜在トピック座標上に次元圧縮している
    - アイデア
        - 文書におけるトピックの確率分布　→　ディリクレ分布　αパラメータ
        - トピックにおける生成される単語の確率分布　→　ディリクレ分布　βパラメータ
        - 上記を使って、潜在変数を推定していく
- 共役事前分布
    - メリット：ベイズ推定は事後分布の式が複雑になりがちだが、共役事前分布によって式自体は変わらないため、計算が楽
            - 一方で、最近のベイズ推定では、 MCMCでベイズ推定が簡単に行えるようにもなってきている。
    - 事前確率と事後確率で変化がない分布
    - ディリクレ分布のパラメータα
        - データを観測する前の潜在変数の仮想的頻度
        - 単語の出現確率で例えると、単語の出現確率の偏りを表す
        - αは大きくすると、ざっくりとしたトピックになり、
        - αを小さくすると、詳細なトピックになる
            - 理由は、ディリクレ分布のパラメータであり、単語の偏りを表しているため
            - αを小さくする　→　単語郡の周辺に集まる　→　意味度合いが強い単語が集まる　→　詳細なトピック
            - αを大きくする　→　単語郡の中心に集まる　→　意味が強くなく単語が集まる　→　ざっくりとしたトピック
        - αが全て同じ場合と、異なる場合で性質が違う
            - αは非対称の方が解釈性という意味においても精度が高い
            - よりちゃんとしたαを知るためには、事後分布のパラメータを周辺化した周辺対数尤度におけるα,βの最適化が必要　→　経験ベイズ
- 学習アルゴリズム
    - カルバックライブラー情報量（相対エントロピー）
        - 統計モデルの分布の近さを表す
        - 一様分布からエントロピーがどの程度離れているのかの指標（一様分布に従う時、最大）
    - ベイズ推定とMAP推定の違い
        - 推定後の結果
            - MAP推定は、最大事後確率をとってくるため点推定
            - ベイズ推定は、事後確率を全部使うため確率分布
        - ベイズ推定は、確率分布が出てからギブスサンプリング等をする必要がある　※積分できない複雑な式のため
    - ベイズ推定から出た事後分布を使って、サンプリングしてパラメータを予測する方法
        - 変分ベイズ
            - アイデア　
                - 今の事後分布は複雑すぎるので、別の確率分布を使って推定するように考える　※出来るだけ似ている分布
                - →　カルバックライブラー推定方を使う　→　分布が近いかどうかの指標
                - 別の確率分布も計算が難しいので、変分下限（周辺対数尤度）を最大化させる
        - ギブスサンプリング
            - 条件付き確率分布を使って、確率変数を交互にサンプル生成し、目的の事後分布をサンプル生成する
        - 周辺化ギブスサプリング
            - 実装が簡単
            - アイデア
                - ギブスサンプリングでは、パラメータのサンプリング　→　求めたい確率のサンプリングを行った。
                - より簡単にするために、パラメータを積分消去してからサンプリング
                - 確率変数を積分消去するため、周辺化という
                - ※通常は積分計算が難しいが、パラメータの確率変数に共役事前分布を用いているので問題なし！


****

## 予測性能
### パープレシキティ：予測性能
- 確率の逆数　→　0.01なら、100個の中から正解が1つあるイメージ
- 理論
	- データの次に入る単語は、文章の成り立ちからある程度推測がつく（名詞 or 動詞）
	- 単語は「入る」か「入らない」ではなく、「確率」で表す
	- perplexity = 1 /P(正解 | M)
	- モデルMの中で、正解語を選ぶ難しさを表す
		- →　候補数が少ないほど正解数を当てやすい　→　予測性能
### コヒーレンス：トピックの品質
- 指標はたくさんあるが、u_massを使用
- トピック自体の分かりやすさを求める
- トピックの頻出単語の共起性が高いほど、コヒーレンスも高くなる
	- →　トピックの特性がよりよくわかるので、人間が理解しやすくなる？
- 各トピックのコヒーレンスの平均がコヒーレンスになる

## チューニング

- BoW
	- レシート単位の購買アイテム　※　同じアイテムが複数出現等は、商品によって偏りが大きい
- α
	- 単語ごとの特徴をより捉えたい　→　小さく設定
- η
	- ？？？　→　とりあえずデフォルト？
- トピック数
	- perplexity / coherenceと解釈できるこちらの限界トピック数を元に判断　→　15 - 30の間